In [1]:
import pandas as pd
import numpy as np
from collections import Counter
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
data = pd.read_csv(r'C:\Users\jespe\Desktop\Simula Summerjob\stati (1).csv', usecols=["id", "text"], encoding="utf8", dtype=str)

In [3]:
#Open files of IDs of tweets to learn from and goes through a dict to remove dups
#Expects csv file, 1 column, just IDs
with open(r'C:\Users\jespe\Desktop\Simula Summerjob\Testing All Reg IDs.csv') as fil:
    IDs_reg = [line.rstrip('\n') for line in fil]
IDs_reg = list(dict.fromkeys(IDs_reg))
with open(r'C:\Users\jespe\Desktop\Simula Summerjob\Testing All Consp IDs.csv') as fil:
    IDs_consp = [line.rstrip('\n') for line in fil]
IDs_consp = list(dict.fromkeys(IDs_consp))
    
#Define Special Characters to remove
#Remove them because the wordcount would consider 5g and 5g: as two different entities
spec_chars = [",", ".", "!", "'", ":", ";", "?", '"', "(", ")", "#"]


#Use IDs and data file to select wanted tweets and text
Tweets_IDs_reg = data.loc[data['id'].isin(IDs_reg)]
tweets_reg = Tweets_IDs_reg['text']

#Delete duplicate tweets, remove special characters, make lower case and give every word its own column
non_dup_reg = tweets_reg.drop_duplicates(keep=False)
#Takes each word into a different column
separate_reg = non_dup_reg.str.split(expand=True)
combined_reg = pd.Series(separate_reg.values.ravel('F'))
#removing the special characters from above
for char in spec_chars:
    combined_reg = combined_reg.str.replace(char, '')
#tranform everthing into lower+case so 5G and 5g become the same    
final_words_reg = combined_reg.str.lower()

#Count how many times each word is mentioned and sort frame with highest mentioned words
counts_reg = Counter(final_words_reg)
#counting the words and returns a dataframe [word, wordCount]
df_reg = pd.DataFrame.from_dict(counts_reg, orient='index').reset_index()
df_reg = df_reg.rename(columns={'index':'Word', 0:'Count Reg'})
df_reg = df_reg.sort_values(by=['Count Reg'], ascending=False)

#Creates ratio of word count per tweet
#In case we have the word 60hz 100 times and 1000 overall tweets we get 1/10
df_reg['Ratio Reg'] = df_reg['Count Reg']/len(non_dup_reg)

#Repeats Process for second set of data 
#The stuff above is for regular tweets while this is the same logic
#targeting the conspiracy tweets
Tweets_IDs_consp = data.loc[data['id'].isin(IDs_consp)]
tweets_consp = Tweets_IDs_consp['text']
non_dup_consp = tweets_consp.drop_duplicates(keep=False)
separate_consp = non_dup_consp.str.split(expand=True)
combined_consp = pd.Series(separate_consp.values.ravel('F'))
for char in spec_chars:
    combined_consp = combined_consp.str.replace(char, '')
final_words_consp = combined_consp.str.lower()
counts_consp = Counter(final_words_consp)
df_consp = pd.DataFrame.from_dict(counts_consp, orient='index').reset_index()
df_consp = df_consp.rename(columns={'index':'Word', 0:'Count Consp'})
df_consp = df_consp.sort_values(by=['Count Consp'], ascending=False)
df_consp['Ratio Consp'] = df_consp['Count Consp']/len(non_dup_consp)


#Merges Conspiracy and Regular tweet counting results
#resulting in a dataframe [word, conspiracyCnt, conspiracyRatio, regularCnt, regularRatio]
result_count = pd.merge(df_consp, df_reg, how='outer', on='Word')

#Creates usage ratio by Consp ratio / Reg Ratio
result_count['Usage Ratio'] = result_count['Ratio Consp'] / result_count['Ratio Reg']
result_count = result_count.sort_values(by=['Usage Ratio'], ascending=False)
df_ratio = result_count[['Word', 'Usage Ratio']]
#drops all usageratio = 0
df_ratio = df_ratio.dropna()

#Puts word and corresponding usage ratio in dictionary
dic = dict(df_ratio.values.tolist())

In [4]:
print(dic)

{'blows': 62.899628252788105, 'mandatory': 43.24349442379182, 'activating': 27.518587360594797, 'plandemic': 23.587360594795538, 'audio': 19.656133828996285, '@worldtruthtv': 19.656133828996285, 'molecule': 19.656133828996285, 'ha': 19.656133828996285, 'switched': 19.656133828996285, 'metals': 19.656133828996285, 'complications': 18.345724907063197, '“coronavirus”': 17.690520446096652, 'october': 17.690520446096652, 'o2': 17.690520446096652, 'deprivation': 15.724907063197028, 'introduced': 15.724907063197026, 'weakening': 15.724907063197026, '£20': 15.724907063197026, 'absorbed': 15.724907063197026, 'syndrome': 15.724907063197026, 'sheeple': 15.724907063197026, 'nanotech': 15.724907063197026, 'martin': 15.724907063197026, 'smokescreen': 15.724907063197026, 'gigahertz': 15.724907063197026, 'connector': 15.724907063197026, 'atmosphere': 15.724907063197026, 'hemoglobin': 15.724907063197024, '5gkills': 15.724907063197024, 'wakeup': 13.759293680297398, 'trigger': 13.759293680297398, 'suscep

In [5]:
#Open files of IDs of tweets to test and goes through a dic to remove dups
#Expects csv file, 1 column, just IDs
with open(r'C:\Users\jespe\Desktop\Simula Summerjob\Testing Target IDs.csv') as fil:
    IDs_all = [line.rstrip('\n') for line in fil]
IDs_all = list(dict.fromkeys(IDs_all))

#Takes tweets from ID list, makes all lower case and removes special characters
data_sort = data.loc[data['id'].isin(IDs_all)]
data_sort['text'] = data_sort['text'].str.lower()
for char in spec_chars:
    data_sort['text'] = data_sort['text'].str.replace(char, '')

#All words in separate column    
split_text = data_sort['text'].str.split(expand=True)
split_text['id'] = data_sort['id']
split_text = split_text.set_index('id').reset_index()

#From df to np.array
array_data = split_text.values

In [6]:
print(array_data)

[['1246120254192996353' 'why' '5g' ... None None None]
 ['1243567367742992386' 'so' 'we' ... None None None]
 ['1241978718320332802' 'wakeup' 'to' ... None None None]
 ...
 ['1250877759431327745' '@doulmak' '@cherriesherie' ... None None None]
 ['1247392928026095616' '@0ctoberreignz' 'i’ve' ... None None None]
 ['1244184285394976773' '@chigaz_paulo' '@rewardkangai' ... None None
  None]]


In [7]:
#Defines function, takes values of a word from the dict, replaces word with value
#and returns product of all values in a tweet. Counter goes up per word counted, prod creates a product of all words in a
#tweet to be used for geometric mean. Sum creates the sum for all words in a tweet to be used for sum and arithmetic mean
def geomean (line, dic):
    prod = 1
    counter = 0
    sum = 0
    for word in line:
        if word != None:
            for key in dic.keys():
                if key == word:
                    sum += dic[key]
                    prod *= dic[key]
                    counter += 1
    if counter == 0:
        counter = 1
    return prod**(1/counter)
    #return sum

#Empty dictionary, puts tweet ID and corresponding geometric mean of values for a tweet in dictionary
dic2 = {}
for line in array_data:
    dic2[line[0]] = geomean(line[1:], dic)

#From dictionary to dataframe
result = pd.DataFrame(dic2.items(), columns=['id', 'Output Value'])

In [8]:
result = result.sort_values(by=['Output Value'], ascending=False)
#result.to_csv('SumTestForFinalNoHash.csv')

In [9]:
print(result)

                       id  Output Value
1102  1247054303107395584      4.174536
1424  1258571667359531009      4.103636
1805  1245599591950475264      3.816425
5086  1244972712298151936      3.678366
3051  1251215426803183616      3.593340
...                   ...           ...
4767  1246620693850013697      0.596396
1624  1251430954616229894      0.556965
3914  1228215960693137408      0.507566
390   1244933430204301315      0.487803
197   1244267179425447936      0.302402

[5311 rows x 2 columns]
